In [3]:
import torch
import torch.nn as nn
import torchvision
from torchvision import datasets
from torch.utils.data import Dataset, DataLoader, random_split
import numpy as np
from torchvision import datasets, transforms
from PIL import Image, ImageOps
import matplotlib.pyplot as plt
import import_ipynb
import glob
import random
import cv2
import imutils
# %run data.ipynb

# Basic U-Net 
Based on https://www.mdpi.com/2073-8994/14/11/2295 
Using https://github.com/richzhang/colorization/blob/master/colorizers/eccv16.py for syntax help

In [4]:
class CUNet(nn.Model):
    def __init__(self, batch = nn.BatchNorm2d):

        # this one downsamples using avg pool idk
        
        # output dim = floor(input dim)
        self.down = torch.nn.AvgPool2d(2) 

        self.conv1_1 = nn.Sequential(
            nn.Conv2d(1, 64, kernel_size = 3, step = 1, padding = 1),
            nn.Sigmoid(),
            nn.BatchNorm2d()
        )

        self.conv1_2 = nn.Sequential(
            nn.Conv2d(64, 64, kernel_size = 3, step = 1, padding = 1),
            nn.Sigmoid(),
            nn.BatchNorm2d()
        )

        #downsample

        self.conv2_1 = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size = 3, step = 1, padding = 1),
            nn.Sigmoid(),
            nn.BatchNorm2d()
        )

        self.conv2_2 = nn.Sequential(
            nn.Conv2d(128, 128, kernel_size = 3, step = 1, padding = 1),
            nn.Sigmoid(),
            nn.BatchNorm2d()
        )

        #downsample

        self.conv3_1 = nn.Sequential(
            nn.Conv2d(128, 256, kernel_size = 3, step = 1, padding = 1),
            nn.Sigmoid(),
            nn.BatchNorm2d()
        )

        self.conv3_2 = nn.Sequential(
            nn.Conv2d(256, 256, kernel_size = 3, step = 1, padding = 1),
            nn.Sigmoid(),
            nn.BatchNorm2d()
        )

        #downsample

        self.conv4_1 = nn.Sequential(
            nn.Conv2d(128, 256, kernel_size = 3, step = 1, padding = 1),
            nn.Sigmoid(),
            nn.BatchNorm2d()
        )

        self.conv4_2 = nn.Sequential(
            nn.Conv2d(128, 256, kernel_size = 3, step = 1, padding = 1),
            nn.Sigmoid(),
            nn.BatchNorm2d()
        )

        
    def forward(self, x):

    

SyntaxError: incomplete input (2804254556.py, line 66)